In [98]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import time, random
import pandas as pd
import json

In [99]:
# 1. 웹 드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)

In [111]:
# 2. 네이버 쇼핑 페이지로 이동
driver.get("https://shopping.naver.com/ns/home")
time.sleep(random.uniform(1, 2))

In [118]:
# 3. 검색창을 찾아 검색어 입력
search_keyword = "탈모"
search_box_before = driver.find_element(By.CSS_SELECTOR, 'input#input_text')
search_box_before.click()

In [ ]:
search_box_before = driver.find_element(By.CSS_SELECTOR, 'input#input_text')
search_box_before.send_keys(search_keyword)
search_box_before.send_keys(Keys.ENTER)
time.sleep(random.uniform(1, 2))

In [124]:
# 4. 페이지 끝까지 스크롤하여 모든 상품 로드
print("페이지 끝까지 스크롤하여 모든 상품을 로드합니다...")
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(1.5, 2.5)) # 페이지 로딩 대기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("스크롤 완료.")
        break
    last_height = new_height

페이지 끝까지 스크롤하여 모든 상품을 로드합니다...
스크롤 완료.


In [125]:
# 5. 최종 로드된 페이지 소스로 soup 객체 생성
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
soup.title.text

'탈모 : 네이버+ 스토어 검색'

In [127]:
# 6. 모든 상품 카드 선택
composite_card_list = soup.select('div.section_section__8wTvh div#composite-card-list ul.compositeCardList_product_list__Ih4JR li.compositeCardContainer_composite_card_container__jr8cb')
print(f"총 {len(composite_card_list)}개의 상품을 수집합니다.")

총 50개의 상품을 수집합니다.


In [87]:
print(composite_card_list[0].select_one('div').prettify())

<div class="basicProductCard_basic_product_card__TdrHT basicProductCard_view_type_grid2__vKr1n">
 <a aria-labelledby="basic_product_card_information_10609048071" class="basicProductCard_link__urzND _nlog_click _nlog_impression_element" data-shp-abt_exps='[{"serId":"shopping","type":"oep","expId":"NNS-ARA","varId":"3","value":{"bucket":"2","is_control":false},"userType":"nnb","provId":"","sesnId":""}]' data-shp-action-uid="6930b6a8ba1544a58e1e81f09354a745" data-shp-area="prod.pd" data-shp-area-dtl='[{"key":"view_type","value":"1"}]' data-shp-area-id="pd" data-shp-area-type="slot" data-shp-contents-dtl='[{"key":"chnl_prod_no","value":"10609048071"},{"key":"nv_mid","value":"88153553661"},{"key":"click_url","value":"https://smartstore.naver.com/main/products/10609048071"},{"key":"price","value":"32000"},{"key":"cat_id","value":"50000304"},{"key":"loungeonly_yn","value":"n"},{"key":"purchase_yn","value":"n"},{"key":"promotion_yn","value":"n"},{"key":"coupontag_yn","value":"n"},{"key":"n_pic

In [88]:
data_string = (composite_card_list[0].select_one('div a')['data-shp-contents-dtl'])
print(data_string)

[{"key":"chnl_prod_no","value":"10609048071"},{"key":"nv_mid","value":"88153553661"},{"key":"click_url","value":"https://smartstore.naver.com/main/products/10609048071"},{"key":"price","value":"32000"},{"key":"cat_id","value":"50000304"},{"key":"loungeonly_yn","value":"n"},{"key":"purchase_yn","value":"n"},{"key":"promotion_yn","value":"n"},{"key":"coupontag_yn","value":"n"},{"key":"n_pick_yn","value":"n"},{"key":"nfa_type","value":"도착보장"},{"key":"nfa_arv_ymd","value":"2025-09-10"},{"key":"prod_nm","value":"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개"}]


In [89]:
data_list = json.loads(data_string)
print(data_list)

[{'key': 'chnl_prod_no', 'value': '10609048071'}, {'key': 'nv_mid', 'value': '88153553661'}, {'key': 'click_url', 'value': 'https://smartstore.naver.com/main/products/10609048071'}, {'key': 'price', 'value': '32000'}, {'key': 'cat_id', 'value': '50000304'}, {'key': 'loungeonly_yn', 'value': 'n'}, {'key': 'purchase_yn', 'value': 'n'}, {'key': 'promotion_yn', 'value': 'n'}, {'key': 'coupontag_yn', 'value': 'n'}, {'key': 'n_pick_yn', 'value': 'n'}, {'key': 'nfa_type', 'value': '도착보장'}, {'key': 'nfa_arv_ymd', 'value': '2025-09-10'}, {'key': 'prod_nm', 'value': '바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개'}]


In [52]:
# 필요한 정보를 담을 변수를 미리 만듭니다.
product_title = ""
product_price = ""
product_link = ""

# 리스트의 각 딕셔너리를 하나씩 확인합니다.
for item in data_list:
    if item['key'] == 'prod_nm':
        product_title = item['value']
    elif item['key'] == 'price':
        product_price = item['value']
    elif item['key'] == 'click_url':
        product_link = item['value']

# 결과를 출력합니다.
print(f"상품 제목 : {product_title}")
print(f"가격 : {product_price}")
print(f"링크 : {product_link}")

상품 제목 : 바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개
가격 : 32000
링크 : https://smartstore.naver.com/main/products/10609048071


In [97]:
print(composite_card_list[50].prettify())

<li class="compositeCardContainer_composite_card_container__jr8cb composite_card_container" style="position: absolute; top: 5922.84px; left: 326px;">
 <div class="basicProductCardSkeleton_basic_product_card_skeleton__Mo7Bb basicProductCardSkeleton_view_type_grid2__rKGa_ basicProductCardSkeleton_delay1__gOTbI">
  <div class="basicProductCardSkeleton_product_card_thumbnail__AFxsz">
   <div class="basicProductCardSkeleton_thumbnail__BVa_I">
   </div>
  </div>
  <div class="basicProductCardSkeleton_information__u8yB5">
   <div class="basicProductCardSkeleton_bar__wGJsC">
   </div>
   <div class="basicProductCardSkeleton_bar__wGJsC">
   </div>
   <div class="basicProductCardSkeleton_bar__wGJsC">
   </div>
   <div class="basicProductCardSkeleton_bar__wGJsC">
   </div>
  </div>
 </div>
</li>



In [90]:
# --- 1. 링크 (Link) ---
link_element  = composite_card_list[4].select_one('a.basicProductCard_link__urzND')['href']
print(link_element)

https://smartstore.naver.com/main/products/9006195975?nl-query=%ED%83%88%EB%AA%A8&nl-au=b7f3133533954de99ba2a06277f2d297&NaPm=ci%3Db7f3133533954de99ba2a06277f2d297%7Cct%3Dmfc8xk7n%7Ctr%3Dnslspp%7Csn%3D386039%7Chk%3Dc85848e3f6773b74e7f78033b6bd16e3a0c0121c


In [91]:
# --- 2. 상품제목 (Product Title) ---
title_element = composite_card_list[4].select_one('strong.productCardTitle_product_card_title__eQupA').text.strip()
print(title_element)

라보에이치 탈모샴푸 약산성 대용량 두피 비건 400ml+400ml리필+180ml


In [92]:
# --- 3. 업체명 (Store Name) ---
store_element = composite_card_list[4].select_one('span.productCardMallLink_mall_name__5oWPw').text.strip()
print(store_element)

아모레퍼시픽몰 헤어바디


In [93]:
# --- 4. 할인전 판매가 (Original Price) ---
original_price_element = composite_card_list[4].select_one('span.priceTag_original_price__jyZRY')
if original_price_element:
    # "할인 전 판매가", "원", 공백을 모두 제거
    original_price = int(original_price_element.text.replace('할인 전 판매가', '').replace('원', '').replace(',', '').strip())
else:
    original_price = "할인전 가격 없음"
print(original_price)

44600


In [94]:
# --- 5. 현재가격 (Current Price) ---
current_price_element = composite_card_list[4].select_one('span.priceTag_number__1QW0R')
if current_price_element:
    current_price = int(current_price_element.text.replace(',', '').strip())
else:
    current_price = "현재가격 없음"
print(current_price)

29800


In [95]:
# --- 6. 할인율 (Discount Rate) ---
discount_element = composite_card_list[4].select_one('span.priceTag_discount__F_ZXz')
if discount_element:
    # "할인", 공백을 모두 제거
    discount_rate = int(discount_element.text.replace('할인', '').replace('%', '').strip())/100
else:
    discount_rate = "할인율 없음"
print(discount_rate)

0.33


In [96]:
link_list, title_list, store_list, original_price_list, current_price_list, discount_rate_list = [], [], [], [], [], []
for i, one in enumerate(composite_card_list):
    # --- 1. 링크 (Link) ---
    link_element  = one.select_one('a.basicProductCard_link__urzND')['href']
    # --- 2. 상품제목 (Product Title) ---
    title_element = one.select_one('strong.productCardTitle_product_card_title__eQupA').text.strip()
    # --- 3. 업체명 (Store Name) ---
    store_element = one.select_one('span.productCardMallLink_mall_name__5oWPw').text.strip()
    # --- 4. 할인전 판매가 (Original Price) ---
    original_price_element = one.select_one('span.priceTag_original_price__jyZRY')
    if original_price_element:
        original_price = int(original_price_element.text.replace('할인 전 판매가', '').replace('원', '').replace(',', '').strip())
    else:
        original_price = "할인전 가격 없음"
    # --- 5. 현재가격 (Current Price) ---
    current_price_element = one.select_one('span.priceTag_number__1QW0R')
    if current_price_element:
        current_price = int(current_price_element.text.replace(',', '').strip())
    else:
        current_price = "현재가격 없음"
    # --- 6. 할인율 (Discount Rate) ---
    discount_element = one.select_one('span.priceTag_discount__F_ZXz')
    if discount_element:
        discount_rate = int(discount_element.text.replace('할인', '').replace('%', '').strip())/100
    else:
        discount_rate = "할인율 없음"

    # 리스트에 추가
    link_list.append(link_element)
    title_list.append(title_element)
    store_list.append(store_element)
    original_price_list.append(original_price)
    current_price_list.append(current_price)
    discount_rate_list.append(discount_rate)
    
    print(i, title_element, store_element, original_price, current_price, discount_rate, link_element)
    print()

0 바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개 바이브랩 할인전 가격 없음 32000 할인율 없음 https://ader.naver.com/v1/6lCNUDBneamHvirMc4ysKHd5CRr_GmpH2DRQwqVL8ZptzM_vmckiYZ0Vg2RU0py-H7knNu9IDG_Z5IQMh6du02kvADikZBRPD-AfPlxBKgWOSzv9shS6mvwz9Jl-PMOBnCK3lDu-aSsC-nWanoFQFHORaGWRbK1YhjLdJP4Hh_tGVMKJXtfilH9sR5A7uIJ7NI6XBMFEHoyZEI1aAvT6Kr3sWYkc-RBo4d9PIcp1ueUXCY-YCMSKBwTb-T-V8EkWtT-Ef29Fi249qs9re5jR64Yegrk4D5CD_fxLViK1b-1HZFFtX2DnObKw5RFmWIsIRrgiKnrOC71UwKFJ-eqE9GcygHB1nJjY6p0pByDMRt67gT7nqfx4Ohs_6XsCOSzvWOLsrcyz-G2UsiHfX3N0Qoz18BOTpaEmvFHrGYU5Rlz9XR4XkHa6VNn0v1GrRv5gUym6E3EYd99BYSrWNeD-UQ==?c=pc.nplusstore.npla&t=0

1 릴리이브 탈모케어 정수리 M자완화 볼륨 샴푸 릴리이브 32000 19800 0.38 https://ader.naver.com/v1/6h2Y1jWtu6TA30XWbFHq7iXR5eOMsSF1XLcG_atHZP75vTpNMDGGx1AXR4KHuGFTt72Y3E8AXm1c9oNZWOuSN0jfV9zQiB614duSOeVVJo7ZT8Tft1x4aqJ02_5XWuCH6mzhRI2Fgla-GtQAzeCUdxWu9M2DAnGrd0N5xoY8a5Wj4OA5nI4hN5OOgfTkF2-bxLwjU9yHF7KKssC47715b3o7h2mI-4uXePLbMFnBIqjTXAgS_PPO1KaVRB0JhRfzRjJWNsDlH0_WSKVXEHnbBNuUDwaixqufraN84gYnRcuE7SrdpwzDSTjrOtxO1uV7vL6bE9CRC

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random

# (드라이버 설정 및 검색어 입력까지의 코드는 동일)
# 1. 웹 드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)

# 2. 네이버 쇼핑 페이지로 이동
driver.get("https://shopping.naver.com/ns/home")
time.sleep(random.uniform(1, 2))

# 3. 검색창을 찾아 검색어 입력
search_keyword = "탈모"
search_box = driver.find_element(By.XPATH, '//*[@id="gnb-gnb"]/div[2]/div/div[2]/div[1]/form/div/div/div/div/input')
search_box.send_keys(search_keyword)
search_box.send_keys(Keys.RETURN)
time.sleep(random.uniform(1, 2))


# ===================================================================
#               수정된 스크롤 및 데이터 수집 로직
# ===================================================================


# 페이지 끝까지 스크롤하여 모든 상품 로드
print("페이지 끝까지 스크롤하여 모든 상품을 로드합니다...")
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(1.5, 2.5)) # 페이지 로딩 대기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("스크롤 완료.")
        break
    last_height = new_height

# 최종 로드된 페이지 소스로 soup 객체 생성
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# 모든 상품 카드 선택
composite_card_list = soup.select('div.section_section__8wTvh div#composite-card-list ul.compositeCardList_product_list__Ih4JR li.compositeCardContainer_composite_card_container__jr8cb')
print(f"총 {len(composite_card_list)}개의 상품을 수집합니다.")

# 최종 데이터를 담을 리스트
link_list, title_list, store_list, original_price_list, current_price_list, discount_rate_list = [], [], [], [], [], []

# 모든 상품 카드 순회하며 데이터 추출
for i, one in enumerate(composite_card_list):
    try:
        # --- 1. 링크 (Link) ---
        link_element = one.select_one('a.basicProductCard_link__urzND')['href']
        
        # --- 2. 상품제목 (Product Title) ---
        title_element = one.select_one('strong.productCardTitle_product_card_title__eQupA').text.strip()
        
        # --- 3. 업체명 (Store Name) ---
        store_element = one.select_one('span.productCardMallLink_mall_name__5oWPw').text.strip()
        
        # --- 4. 할인전 판매가 (Original Price) ---
        original_price_tag = one.select_one('span.priceTag_original_price__jyZRY')
        original_price = int(original_price_tag.text.replace('할인 전 판매가', '').replace('원', '').replace(',', '').strip()) if original_price_tag else 0

        # --- 5. 현재가격 (Current Price) ---
        current_price_tag = one.select_one('span.priceTag_number__1QW0R')
        current_price = int(current_price_tag.text.replace(',', '').strip()) if current_price_tag else 0

        # --- 6. 할인율 (Discount Rate) ---
        discount_tag = one.select_one('span.priceTag_discount__F_ZXz')
        discount_rate = int(discount_tag.text.replace('할인', '').replace('%', '').strip()) / 100 if discount_tag else 0.0

        # 리스트에 추가
        link_list.append(link_element)
        title_list.append(title_element)
        store_list.append(store_element)
        original_price_list.append(original_price)
        current_price_list.append(current_price)
        discount_rate_list.append(discount_rate)
        
        print(f"✅ [{i+1}/{len(composite_card_list)}] 수집 성공: {title_element}")

    except Exception as e:
        print(f"❌ [{i+1}/{len(composite_card_list)}] 수집 중 오류 발생. 건너뜁니다. 오류: {e}")
        # 오류 발생 시에도 리스트 길이를 맞추기 위해 실패 값을 넣어주는 것이 좋습니다.
        link_list.append("수집실패")
        title_list.append("수집실패")
        store_list.append("수집실패")
        original_price_list.append(0)
        current_price_list.append(0)
        discount_rate_list.append(0)

# 드라이버 종료
driver.quit()

In [128]:
# 최종 데이터를 담을 리스트
link_list, title_list, store_list, original_price_list, current_price_list, discount_rate_list = [], [], [], [], [], []

# 모든 상품 카드 순회하며 데이터 추출
for i, one in enumerate(composite_card_list):
    try:
        # --- 1. 링크 (Link) ---
        link_element = one.select_one('a.basicProductCard_link__urzND')['href']
        
        # --- 2. 상품제목 (Product Title) ---
        title_element = one.select_one('strong.productCardTitle_product_card_title__eQupA').text.strip()
        
        # --- 3. 업체명 (Store Name) ---
        store_element = one.select_one('span.productCardMallLink_mall_name__5oWPw').text.strip()
        
        # --- 4. 할인전 판매가 (Original Price) ---
        original_price_tag = one.select_one('span.priceTag_original_price__jyZRY')
        original_price = int(original_price_tag.text.replace('할인 전 판매가', '').replace('원', '').replace(',', '').strip()) if original_price_tag else 0

        # --- 5. 현재가격 (Current Price) ---
        current_price_tag = one.select_one('span.priceTag_number__1QW0R')
        current_price = int(current_price_tag.text.replace(',', '').strip()) if current_price_tag else 0

        # --- 6. 할인율 (Discount Rate) ---
        discount_tag = one.select_one('span.priceTag_discount__F_ZXz')
        discount_rate = int(discount_tag.text.replace('할인', '').replace('%', '').strip()) / 100 if discount_tag else 0.0

        # 리스트에 추가
        link_list.append(link_element)
        title_list.append(title_element)
        store_list.append(store_element)
        original_price_list.append(original_price)
        current_price_list.append(current_price)
        discount_rate_list.append(discount_rate)
        
        print(f"✅ [{i+1}/{len(composite_card_list)}] 수집 성공: {title_element}")

    except Exception as e:
        print(f"❌ [{i+1}/{len(composite_card_list)}] 수집 중 오류 발생. 건너뜁니다. 오류: {e}")
        # 오류 발생 시에도 리스트 길이를 맞추기 위해 실패 값을 넣어주는 것이 좋습니다.
        link_list.append("수집실패")
        title_list.append("수집실패")
        store_list.append("수집실패")
        original_price_list.append(0)
        current_price_list.append(0)
        discount_rate_list.append(0)

✅ [1/50] 수집 성공: 해외퓨루 리시리 컬러 샴푸 라이트 브라운 200ml
✅ [2/50] 수집 성공: 해외펜테 포르테 프로 서브리믹 샴푸 OS 500ml
✅ [3/50] 수집 성공: 스파 헤드 마사지기 안마기
✅ [4/50] 수집 성공: 해외LIPPS 립스 하드 헤어 스프레이 200g
✅ [5/50] 수집 성공: 해외Aujua 오주아 DIORUM 디오럼 모이스트캄 트리트먼트 1000g
✅ [6/50] 수집 성공: 해외아드반젠 아트리스 하이브리드 스칼프 로션 100mL
✅ [7/50] 수집 성공: 해외덴마크 Scandinavian 남자용 모발 성장 루틴 Hair Growth-Routine
✅ [8/50] 수집 성공: 해외펜테 포르테 세범 클리어 젤 쿨 150g
✅ [9/50] 수집 성공: 해외FURORIN 약용 두피용 에센스 180mL (3개세트)
✅ [10/50] 수집 성공: 해외시스템 4 바이오 보태니컬 샴푸 500ml/ 세럼 500ml
✅ [11/50] 수집 성공: 해외토픽 헤어 빌딩 파이버스 흑채 12g 블랙
✅ [12/50] 수집 성공: 해외Mroobest 안티 헤어 로스 샴푸 K4 220Ml
✅ [13/50] 수집 성공: 해외아쥬반 리 에미사리 샴푸 트리트먼트 세트 리필용
✅ [14/50] 수집 성공: 두피 헤어 마사지 샴푸 브러쉬
✅ [15/50] 수집 성공: 해외제노리스 피스타치오 헤어 트리트먼트 오일 100ml
✅ [16/50] 수집 성공: 해외마브 하이엘 G 샴푸 300mL
✅ [17/50] 수집 성공: EM 스칼프 두피건강 모발 두피 헤어샴푸 500ml
✅ [18/50] 수집 성공: 해외티오신(thiocyn) 여성용 헤어 세럼 150ml
✅ [19/50] 수집 성공: 비욘드 힐링포스 스캘프 클리닉 스케일러 100ml
✅ [20/50] 수집 성공: 두피마사지기 프로 헤어케어 두피 마사지기
✅ [21/50] 수집 성공: 해외펜테 포르테 프로페셔널 트리트먼트 a 리필 450g
✅ [22/50] 수집 성공: 프리미엄TS샴푸300g + 프리미엄

In [129]:
# 드라이버 종료
driver.quit()